# Projeto - Estabilização de Imagens

Gabriel Moreira

### Entrega: 12/Set 23:59 via GitHub.

### Rubrica do Projeto:

    I. Não entregou ou entregou apenas um rascunho.
    D. O programa faz apenas uma estabilização parcial com os programas da Aula 09.
    C. Utiliza o Dense Optical Flow de uma janela no centro da imagem.
    B. Utiliza meios para compensar as faixas pretas nos cantos da imagem com algum limite.
    A. Consegue realizar a compensação em rotação no eixo de profundidade.
    
    +1/2 Conceito para implementações que comprovadamente melhoram o desempenho da estabilização.
    -1/2 Conceito se o notebook não contiver uma explicação detalhada da solução apresentada.

## Solução para estabilização de imagens usando o Dense Optical Flow

**1º passo:** Captura de uma primeira imagem da webcam para servir como base

**2º passo:** Definição de uma área da imagem que será usada no Dense Optical Flow (o que aumentará o desempenho sendo que o algoritmo não terá que percorrer e analisar a imagem inteira). Com isso, espera-se que a imagem da câmera será estabilizada de acordo com o conteúdo dentro da área proposta.

**3º passo:** Uma abordagem possível para determinar o flow do objeto(dentro da área) é calcular a média do flow de todos os pontos que se encontram dentro da área, tanto no eixo X como no eixo Y.

**4º passo:** Uma forma de compensar as eventuais oscilações na câmera seria fazendo o corte na imagem de acordo com o fluxo ótico. Se o fluxo ótico médio de uma área no eixo X é positivo, corta-se uma parte equivalente à direita da imagem por exemplo.

**5º passo:** Uma vez que o corte da imagem em X e Y não obedece a proporção da câmera (aspect ratio), um corte adicional deve ser feito (em X ou Y dependendo do caso) para manter a proporção.

In [ ]:
import cv2 as cv
import numpy as np
import math

captura = cv.VideoCapture(0)

# Para nao deixar encavalar os frames
captura.set(cv.CAP_PROP_BUFFERSIZE, 1)

ret, frame = captura.read()
rows, cols, r = frame.shape
#cols -> x
#rows -> y

#essa mascara funciona na camera do computador (1080, 720)
y1, y2 = int(0.34 * rows), rows - int(0.34 * rows)
# y1, y2 = 200, rows - 200
x1, x2 = int(0.42 * cols), cols - int(0.42 * cols)
# x1, x2 = 480, cols - 480

print(rows, cols)
previous = frame[y1:y2, x1:x2]
previous_gray = cv.cvtColor(previous, cv.COLOR_BGR2GRAY)

maX = 0
maY = 0

while(1):
    ret, frame = captura.read()
    actual = frame
    actual = actual[int(y1):int(y2), int(x1):int(x2)]
    actual_gray = cv.cvtColor(actual, cv.COLOR_BGR2GRAY)
    #calcula o fluxo X e Y
    flow = cv.calcOpticalFlowFarneback(previous_gray, actual_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    #média do fluxos X e Y
    fx = np.mean(flow[:,:,0])
    fy = np.mean(flow[:,:,1])
    
    #soma do fluxo com os fluxos anteriores
    maX += fx
    maY += fy
    
    #limita a mascara no tamanho da imagem
    y1 = min(rows, max(0, y1 + fy))
    y2 = min(rows, max(0, y2 + fy))
    x1 = min(cols, max(0, x1 + fx))
    x2 = min(cols, max(0, x2 + fx))
    M = np.array([[1, 0, -maX], [0, 1, -maY]], dtype=np.float32)
    dst = cv.warpAffine(frame, M, (cols, rows))
    
    #calcula o tamanho da imagem cortada
    yl = rows - np.abs(int(maY))
    xl = cols - np.abs(int(maX))

    if(maY > 0):
        #cortar em baixo
        cy2 = int(rows - maY)
        cy1 = 0
        
    else:
        cy1 = int(-maY)
        cy2 = rows
        # cortar em cima
    if (maX > 0):
        cx2 = int(cols - maX)
        cx1 = 0
        # cortar na direita
    else:
        cx1 = int(-maX)
        cx2 = cols
        #cortar na esquerda

    cY = 0
    cX = 0
    
    #ajustar o aspect ratio
    if(16/9 > xl/yl):
        cY = ((-9/16) * xl) + yl
    
    else:
        cX = ((-16/9) * yl) + xl
    
    #corte da imagem
    dst = dst[cy1:cy2 - int(cY), cx1:cx2 - int(cX)]
    
    #desenha o retângulo
    cv.rectangle(dst,(int(x1),int(y1)), (int(x2),int(y2)), (0,0,255), 3)
    
    dst = cv.resize(dst, (cols, rows), interpolation = cv.INTER_CUBIC) 
    cv.imshow("Video", dst)
    previous_gray = actual_gray.copy()
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break

captura.release()
cv.destroyAllWindows()